In [1]:
import sys
sys.path.insert(0, '/home/serhii/AccountEye/')

import ImagePreprocessing.ImageProcessing as IP
import Experiments.Experiment as E
import Experiments.Models as M
import Experiments.Data as D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as K
import tensorflow as tf

import numpy as np
import cv2


In [2]:
csvPaths = ['Images/Labels/Clean.csv', 
            'Images/Labels/YouTube.csv',
            'Images/Labels/NewBalanced.csv'] 
            
imgPath = 'Images/Resized/'

#csvPaths = ['/home/serhii/AccountEyeTest/Images/Augmented/NewBalanced_Blur+Augmentation.csv',]
#imgPath = '/opt/AccountEye/Images/Preproc/'

trainPart = 1
testPart = 0
batchSize = 256
numClasses = 20
epochs = 10
rows, cols, channels = 48, 32, 3
input_shape = (rows, cols, channels)



In [3]:
(xTrain, yTrainO), (_, _), (_, _) = D.LoadData(csvPaths, imgPath, rows, cols, channels=channels,
                                               trainPart=trainPart, testPart=testPart,
                                               balanceDS=False, labelRow='20_Classes')


In [4]:
yTrain = K.utils.to_categorical(yTrainO, numClasses)

In [19]:
def Generator(features, labels, batchSize, processingFunction, imageGenerator = None):
    if imageGenerator is None:
        imageGenerator = ImageDataGenerator(
            rotation_range=30,
            width_shift_range=0,
            height_shift_range=0,
            rescale=1. / 255,
            shear_range=20, # поворот, сдвиг, растяжение
            zoom_range=0.2,
            brightness_range=(0.7, 1.3), # 0.7 - 1.3
            fill_mode='nearest',
            cval=0)
    while True:
        # Random indexes 
        index = np.array(np.random.choice(features.shape[0]-1, batchSize, replace=False))
        
        # Augmentation batch
        for batch in imageGenerator.flow(features[index], batch_size=batchSize, shuffle=False):
            batchFeatures = batch
            break
        batchLabels = labels[index]
        
        
        for i in range(batchSize):
            batchFeatures[i] =  processingFunction(batchFeatures[i])
        yield batchFeatures, batchLabels
